In [38]:
import os
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
import librosa
import numpy as np
import IPython.display as ipd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# %matplotlib notebook 
%matplotlib inline

In [89]:
def get_data_from_AudioSegment(raw_data):
    # 返回AudioSegment对象源数据
    return np.fromstring(raw_data, dtype=np.int16)  # 从字符串转换

演示单个案例

In [86]:
test = AudioSegment.from_wav("../../数据集收集/中国科学技术大学_新冠肺炎声音诊断挑战赛/初赛训练集/cough/Positive/305_Positive_male_25_cough.wav")
ipd.Audio(get_data_from_AudioSegment(test.raw_data),rate=test.frame_rate)
# test
len(test)
chunks = split_on_silence(test,
        # must be silent for at least half a second,沉默半秒
        min_silence_len=500,
        # consider it silent if quieter than -16 dBFS
        silence_thresh=-35,  # 分段最大分贝不低于35
        keep_silence=300
        )
print('原分段:', len(chunks))
chunks

C:\Users\86181\AppData\Local\Temp/ipykernel_20708/2888547044.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(raw_data, dtype=np.int16)  # 从字符串转换


9557

原分段: 4


In [87]:
for i, chunk in enumerate(chunks):
    ipd.Audio(get_data_from_AudioSegment(chunks[i].raw_data),rate=test.frame_rate)

C:\Users\86181\AppData\Local\Temp/ipykernel_20708/2888547044.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(raw_data, dtype=np.int16)  # 从字符串转换


In [81]:
# len(chunks[0]) # 查看时间
for i in list(range(len(chunks)))[::-1]:  # 逆序排序
#     print(len(chunks[i]))
    if len(chunks[i]) <= 600 or len(chunks[i]) >= 10000:
        chunks.pop(i)
print('取有效分段(大于0.6s小于10s):', len(chunks))
# os.mkdir("../../数据集收集/中国科学技术大学_新冠肺炎声音诊断挑战赛/初赛训练集/cough/Positive/spilt/",)
for i, chunk in enumerate(chunks):
    chunk.export("../../数据集收集/中国科学技术大学_新冠肺炎声音诊断挑战赛/初赛训练集/cough/Positive/spilt/" +
                 "1133_Positive_male_49_cough"+"_chunk{0}.wav".format(i), format="wav")

取有效分段(大于0.6s小于10s): 4


<_io.BufferedRandom name='../../数据集收集/中国科学技术大学_新冠肺炎声音诊断挑战赛/初赛训练集/cough/Positive/spilt/1133_Positive_male_49_cough_chunk0.wav'>

<_io.BufferedRandom name='../../数据集收集/中国科学技术大学_新冠肺炎声音诊断挑战赛/初赛训练集/cough/Positive/spilt/1133_Positive_male_49_cough_chunk1.wav'>

<_io.BufferedRandom name='../../数据集收集/中国科学技术大学_新冠肺炎声音诊断挑战赛/初赛训练集/cough/Positive/spilt/1133_Positive_male_49_cough_chunk2.wav'>

<_io.BufferedRandom name='../../数据集收集/中国科学技术大学_新冠肺炎声音诊断挑战赛/初赛训练集/cough/Positive/spilt/1133_Positive_male_49_cough_chunk3.wav'>

正则表达式模式是用来匹配以".wav"结尾的字符串，并将".wav"之前的任意字符作为匹配结果。其中"{1,}"表示匹配前面的任意字符至少1次，"."表示匹配任意字符，"()"表示匹配结果的组，迭代处理 positive 列表中的每个音频文件

封装函数进行语音分割

In [112]:
def split_sound(file_dir):
    """
    :describe: split on silence and save in new file
    :param: file_dir :watch the back have "/"
    """
    avg_len = []  # 分割后的平均长度
    n = []  # 无效切割音频
    lists = os.listdir(file_dir)
    for name in lists:
        print(name)
        x = re.findall("(.{1,}).wav", name)[0]
#         print(x)  # 文件名前缀
        sound = AudioSegment.from_file(file_dir + name, format="wav")
        chunks = split_on_silence(sound,
        # must be silent for at least half a second,沉默半秒
        min_silence_len=500,
        # consider it silent if quieter than -16 dBFS
        silence_thresh=-35,  # 分段最大分贝不低于35
        keep_silence=300
        )
        print('原分段:', len(chunks))
        # len(chunks[0]) # 查看时间
        for i in list(range(len(chunks)))[::-1]:  # 逆序排序
                if len(chunks[i]) <= 600 or len(chunks[i]) >= 10000:
                    chunks.pop(i)
        print('取有效分段(大于0.6s小于10s):', len(chunks))
        if len(chunks) == 0:
            n.append(name)
        if not os.path.exists(file_dir + "split/"):
            os.mkdir(file_dir + "split/")
        for i, chunk in enumerate(chunks):
            chunk.export(file_dir+"split/" + x +"_chunk{0}.wav".format(i), format="wav")
            avg_len.append(len(chunks[i]))
    print("平均长度",sum(avg_len)/len(avg_len))
    print('完全无效:',n)

In [111]:
split_sound("../../数据集收集/中国科学技术大学_新冠肺炎声音诊断挑战赛/初赛训练集/cough/Positive/")

1008_Positive_male_32_cough.wav
1008_Positive_male_32_cough
原分段: 2
取有效分段(大于0.6s小于10s): 2
1021_Positive_male_25_cough.wav
1021_Positive_male_25_cough
原分段: 6
取有效分段(大于0.6s小于10s): 4
1041_Positive_male_17_cough.wav
1041_Positive_male_17_cough
原分段: 1
取有效分段(大于0.6s小于10s): 1
1044_Positive_female_52_cough.wav
1044_Positive_female_52_cough
原分段: 3
取有效分段(大于0.6s小于10s): 3
1071_Positive_male_25_cough.wav
1071_Positive_male_25_cough
原分段: 3
取有效分段(大于0.6s小于10s): 3
1092_Positive_male_30_cough.wav
1092_Positive_male_30_cough
原分段: 3
取有效分段(大于0.6s小于10s): 3
1122_Positive_female_24_cough.wav
1122_Positive_female_24_cough
原分段: 3
取有效分段(大于0.6s小于10s): 3
1131_Positive_female_22_cough.wav
1131_Positive_female_22_cough
原分段: 3
取有效分段(大于0.6s小于10s): 3
1133_Positive_male_49_cough.wav
1133_Positive_male_49_cough
原分段: 1
取有效分段(大于0.6s小于10s): 1
1166_Positive_female_20_cough.wav
1166_Positive_female_20_cough
原分段: 3
取有效分段(大于0.6s小于10s): 3
1183_Positive_female_48_cough.wav
1183_Positive_female_48_cough
原分段: 5
取有效分段(大于0.6s小于10s): 5
1